# **Classificador Quântico Variacional de Imagens Reais e Imagens Geradas por IA**

- Gabriel Alves Gadelha de Melo
- Lucas Emanuel Sabino de Souza Lima

# **Introdução**

Nos últimos anos, a evolução dos modelos generativos de inteligência artificial, como GANs e Diffusion Models, tornou cada vez mais difícil distinguir imagens artificiais de imagens reais. Essa crescente complexidade levanta preocupações em diversas áreas, como segurança digital, jornalismo, arte e verificação de autenticidade. Nesse contexto, o presente trabalho propõe e compara duas abordagens distintas para a detecção automática de imagens geradas por IA: uma abordagem clássica, baseada em descritores manuais e redes neurais profundas, e uma abordagem híbrida, que incorpora circuitos quânticos variacionais para a tarefa de classificação binária.

A primeira abordagem utiliza algoritmos e tratamento de imagens para alimentar o circuito variacional. Já a segunda, utiliza a extração de características visuais como entropia, cor, textura e componentes de Fourier, combinadas com arquiteturas como CNNs e Transformers para realizar a distinção entre imagens reais e artificiais. O objetivo principal deste projeto é analisar a viabilidade e o desempenho desses dois paradigmas, explorando o potencial da computação quântica como alternativa ou complemento a técnicas clássicas de aprendizado de máquina.

# **Pré-Processamento**

Dataset utilizado: [Link do Kaggle](https://www.kaggle.com/datasets/osmankagankurnaz/dataset-of-ai-generated-fruits-and-real-fruits)

## Padronização das Imagens

In [ ]:
import os
import cv2

# Caminho das imagens de entrada e saída
input_folder = "image_dataset"
output_folder = "apple_dataset"

# Tamanho novo
new_size = (256, 256)

# Criar a pasta de saída, se não existir
os.makedirs(output_folder, exist_ok=True)

def save_and_rename_imgs(prefix):
    # Iterar sobre as imagens
    folder = os.path.join(input_folder, prefix)
    for id, filename in enumerate(os.listdir(folder)):
        if filename.lower().endswith((".jpg")):
            img_path = os.path.join(folder, filename)

            # Ler imagem
            img = cv2.imread(img_path)

            if img is None:
                print(f"Unable to open: {img_path}")
                continue

            # Redimensionar imagem
            resized_img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

            # Renomear a imagem
            new_name = f"{prefix}_apple{id}.jpg"
            output_path = os.path.join(output_folder, prefix, new_name)

            # Salvar a imagem
            cv2.imwrite(output_path, resized_img)

            print(f"Saved to: {output_path}")

if __name__ == "__main__":
    save_and_rename_imgs("ai")
    save_and_rename_imgs("real")
    print("All images saved and renamed")

## Cálculo de Parâmetros

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from scipy.stats import entropy, circstd
from sklearn.preprocessing import MinMaxScaler

def get_image_params(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Image not found: {image_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude = np.abs(fshift)

    media_magnitude = np.mean(magnitude)
    energia_total = np.sum(magnitude**2)
    maximo = np.max(magnitude)
    desvio_padrao = np.std(magnitude)

    mag_norm = magnitude.flatten()
    mag_norm = mag_norm / np.sum(mag_norm)
    entropia = entropy(mag_norm + 1e-8)

    # Redimensiona para 64x64 para manter padrão
    new_img = cv2.resize(img, (64, 64))

    # HLS para hue, lightness, saturation
    hls = cv2.cvtColor(new_img, cv2.COLOR_BGR2HLS)
    H = hls[..., 0].astype(np.float32).flatten() * 2  # Hue em 0-360
    L = hls[..., 1].astype(np.float32).flatten()
    S = hls[..., 2].astype(np.float32).flatten()

    # Chroma a partir do RGB
    B, G, R = cv2.split(new_img)
    B = B.astype(np.float32).flatten()
    G = G.astype(np.float32).flatten()
    R = R.astype(np.float32).flatten()

    max_rgb = np.maximum(np.maximum(R, G), B)
    min_rgb = np.minimum(np.minimum(R, G), B)
    C = max_rgb - min_rgb  # Chroma

    std_H = circstd(H, high=360, low=0)
    std_L = np.std(L)
    std_C = np.std(C)
    std_S = np.std(S)

    return {
        "fft_mean": media_magnitude,
        "fft_total_energy": energia_total,
        "fft_max": maximo,
        "fft_std": desvio_padrao,
        "fft_entropy": entropia,
        "std_hue": std_H,
        "std_chroma": std_C,
        "std_lightness": std_L,
        "std_saturation": std_S
    }

# Salva os parâmetros no arquivo params-dataset.csv
if __name__ == "__main__":
    results = {
        "fft_mean": [],
        "fft_total_energy": [],
        "fft_max": [],
        "fft_std": [],
        "fft_entropy": [],
        "std_hue": [],
        "std_chroma": [],
        "std_lightness": [],
        "std_saturation": [],
        "ai_gen": []
    }

    for i in range(150):
        img_path = os.path.join("apple_dataset", "ai", f"ai_apple{i}.jpg")
        params = get_image_params(img_path)
        if params is not None:
            for key in params:
                results[key].append(params[key])
            results["ai_gen"].append(1)

    for i in range(156):
        img_path = os.path.join("apple_dataset", "real", f"real_apple{i}.jpg")
        params = get_image_params(img_path)
        if params is not None:
            for key in params:
                results[key].append(params[key])
            results["ai_gen"].append(0)

    df = pd.DataFrame(results)

    scaler = MinMaxScaler(feature_range=(0, 100))
    cols_to_normalize = [col for col in df.columns if col != "ai_gen"]
    df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])

    CSV_PATH = "params-dataset.csv"
    df.to_csv(CSV_PATH, index=False)

    print(f"Parameters saved to {CSV_PATH}")

# **Model V-1**




## Download do Dataset Pré-Processado


Antes de executar o modelo, é recomendado baixar o arquivo .zip do dataset já pré-processado usando o seguinte link: [apple-dataset](https://drive.google.com/drive/u/2/folders/1rX2wiOHdqUQEFnnoxN2qCeXnlNnTUViD)

Após baixar o arquivo compactado, faça upload dele para a sua sessão do Colab e em seguida execute a célula abaixo.

In [ ]:
!unzip apple_dataset.zip

Se não ocorreu nenhum erro e foi criado o diretório apple_dataset na sua sessão do Colab, você está pronto para executar o modelo!

## Carregamento e Decomposição das Imagens

Antes de enviar as imagens para o circuito quântico, devemos reduzir as suas dimensões, já que seria computacionalmente inviável passá-las para ele como uma matriz de $256\times256$, por exemplo. Por causa disso, usamos o Principal Component Analysis (PCA), uma técnica que, de forma geral, decompõe a imagem em apenas as suas componentes mais representativas. Com isso, podemos usar esse algoritmo para reduzir as imagens para vetores de 8 dimensões, sendo cada dimensão posteriormente correspondente a um qubit.

In [ ]:
import os
import cv2
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [ ]:
def load_images(dir):
  vectors = []
  for file in os.listdir(dir):
    img_path = os.path.join(dir, file)
    img = cv2.imread(img_path)
    if img is None:
      print(f"Image '{img_path}' not found")
      continue
    img = cv2.resize(img, (32, 32))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Transformação das matrizes das imagens em um vetor
    img_vector = img_rgb.flatten()
    vectors.append(img_vector)
  return np.array(vectors)

In [ ]:
# Definição dos paths das imagens
AI_PATH = os.path.join("apple_dataset", "ai")
REAL_PATH = os.path.join("apple_dataset", "real")

# Carreganento das imagens reais e geradas por IA
real_imgs = load_images(REAL_PATH)
ai_imgs = load_images(AI_PATH)

# Junta os dados
X_total = np.vstack([real_imgs, ai_imgs])

# Rotulação dos dados
y_real = np.zeros(len(real_imgs))
y_ai = np.ones(len(ai_imgs))
y_total = np.concatenate([y_real, y_ai])

# PCA para decompor as imagens em 8 componentes
pca = PCA(n_components=8)
X_pca = pca.fit_transform(X_total)

# Normalização [0, π] para condizer com as rotações do circuito
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_norm = scaler.fit_transform(X_pca)

print("Normalized Images Shape:", X_norm.shape) # 306 imagens de 8 componentes

Normalized Images Shape: (306, 8)


## Circuito Quântico Variacional



Agora que os dados estão prontos para serem inseridos no circuito, podemos definir o circuito variacional propriamente dito.

Caso o *PennyLane* não esteja instalado na sua sessão do Colab, execute a célula abaixo.

In [ ]:
!pip install pennylane

In [ ]:
import torch
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import StronglyEntanglingLayers, BasicEntanglerLayers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, StratifiedKFold
import matplotlib.pyplot as plt

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


In [ ]:
NUM_QUBITS = 8
dev = qml.device("default.qubit", wires=NUM_QUBITS)

NUM_LAYERS_BASIC = 1
NUM_LAYERS_STRONGLY = 1

# Inicialização aleatório dos pesos
weights_basic = torch.nn.Parameter(torch.randn(NUM_LAYERS_BASIC, NUM_QUBITS) * np.pi)
weights_strongly = torch.nn.Parameter(torch.randn(NUM_LAYERS_STRONGLY, NUM_QUBITS, 3) * np.pi)

optimizer = torch.optim.Adam([weights_basic, weights_strongly], lr=0.1)

# Definição do CQV
@qml.qnode(dev, interface="torch")
def circuit(X, weights_basic, weights_strong):
  qml.AngleEmbedding(X, wires=range(NUM_QUBITS), rotation="X")
  BasicEntanglerLayers(weights_basic, wires=range(NUM_QUBITS))
  StronglyEntanglingLayers(weights_strongly, wires=range(NUM_QUBITS))
  return qml.expval(qml.PauliZ(0))

# Função de custo a ser minimizada = Hinge Loss
def hinge_loss(y_pred, y_true):
  losses = torch.clamp(1 - y_true * y_pred, min=0)
  return torch.mean(losses)

### Embedding

Executando uma validação cruzada nos dois modelos (embedding na amplitude com 3 qubits e embedding no ângulo em RY com 8 qubits), foram obtidas as seguintes métricas:

<table>
  <tr>
    <th>Embedding</th>
    <th>Acurácia Média</th>
    <th>Desvio Padrão Acurácia</th>
    <th>Precisão Média</th>
    <th>Recall Média</th>
    <th>F1-Score Média</th>
  </tr>
  <tr>
    <td>Amplitude</td>
    <td>0.928</td>
    <td>0.037</td>
    <td>0.945</td>
    <td>0.907</td>
    <td>0.925</td>
  </tr>
  <tr>
    <td>Ângulo</td>
    <td>0.971</td>
    <td>0.030</td>
    <td>1.000</td>
    <td>0.940</td>
    <td>0.968</td>
  </tr>
</table>

Como é possível observar, o modelo com embedding no ângulo performa melhor em todas as métricas, apesar de possuir um tempo de treinamento maior devido ao seu número de qubits. Por causa disso, optamos pelo Embedding no ângulo para o circuito final. Entretanto, se estivessemos trabalhando com um conjunto de dados mais complexo e precisassemos de mais features das imagens, o embedding na amplitude poderia ser mais interessante por sua maior escalabilidade.

Ainda sobre o embedding no ângulo, fizemos testes com rotações em todos os eixos, mas a RX foi a que melhor performou.

<table>
  <tr>
    <th>Eixo de Rotação</th>
    <th>Acurácia Média</th>
    <th>Desvio Padrão Acurácia</th>
    <th>Precisão Média</th>
    <th>Recall Média</th>
    <th>F1-Score Média</th>
  </tr>
  <tr>
    <td>RX</td>
    <td>0.974</td>
    <td>0.022</td>
    <td>0.987</td>
    <td>0.960</td>
    <td>0.973</td>
  </tr>
  <tr>
    <td>RY</td>
    <td>0.971</td>
    <td>0.030</td>
    <td>1.000</td>
    <td>0.940</td>
    <td>0.968</td>
  </tr>
</table>

Apesar da RY ter uma maior precisão, notamos um certo desbalanceamento nas suas previsões, já que sempre que o modelo parecia muito conservador, apenas prevendo que a imagem era gerada por IA se tivesse "certeza absoluta", tendendo sempre a prever que a imagem era real se tivesse "um pouco de dúvida". O RZ não se encontra na tabela, pois, com ele, o modelo nunca previa que a imagem era gerada por IA, portanto descartamos a possibilidade de usá-lo rapidamente.


### Layers
Após algumas pesquisas, encontramos que os templates de layers `BasicEntanglerLayers` e `StronglyEntanglingLayers` do *PennyLane* eram boas escolhas para classificação. Por causa disso, testamos várias combinações dos dois, e em um treinamento com tamanho de batch igual a 32 e 20 epochs conseguimos as seguintes acurácias:

<table>
  <tr>
    <th></th>
    <th>BasicEntanglerLayers</th>
    <th>StronglyEntanglingLayers</th>
    <th>Acurácia</th>
  </tr>
  <tr>
    <td>Teste 1</td>
    <td>0</td>
    <td>1</td>
    <td>0.968</td>
  </tr>
  <tr>
    <td>Teste 2</td>
    <td>0</td>
    <td>2</td>
    <td>0.984</td>
  </tr>
    <tr>
    <td>Teste 3</td>
    <td>0</td>
    <td>3</td>
    <td>1.000</td>
  </tr>
    <tr>
    <td>Teste 4</td>
    <td>3</td>
    <td>0</td>
    <td>0.645</td>
  </tr>
    <tr>
    <td>Teste 5</td>
    <td>1</td>
    <td>1</td>
    <td>1.000</td>
  </tr>
</table>

Como o Teste 5 desempenhou tão bem quanto o Teste 3, optamos por usar as layers do Teste 5, já que o modelo ficava mais simples e o treinamento era mais rápido. Além disso, testamos também um `BasicEntanglerLayers` após o `StronglyEntanglingLayers`, mas não performou tão bem quanto a ordem que está na tabela, alcançando uma acurácia de $95.2\%$.


### Otimizador

Foram testados os otimizadores Adam e Nesterov usando as suas respectivas implementações do *PyTorch*.
Foram realizados treinamentos sob as mesmas circunstânceas com ambos (tamanho de batch igual a 32 e 20 epochs) e usando o circuito com as layers escolhidas anteriormente, de modo que o Adam atingiu $100\%$ e o Nesterov, $91.9\%$. Por causa disso, optamos pelo otimizador Adam.

### Loss

Inicialmente usamos a função de MSE (Mean Squared Error) como função de loss e a acurácia final atingia cerca de $60\%$ a $70\%$. Entretanto, após algumas pesquisas, encontramos que essa não era uma boa função para classificadores, sendo mais adequada para modelos de regressão. Portanto, optamos por testar a Hinge Loss e a Binary Cross Entropy, que eram mais adequadas para o nosso propósito. Feitos os testes sob as mesmas circunstâncias, obtivemos acurácias idênticas para as duas funções ($98.4\%$) e optamos por usar a Hinge Loss, já que o treinamento era mais rápido com ela.

## Treinamento e Avaliação do Modelo

Para a avaliação final do modelo, optamos por realizá-la usando validação cruzada, já que, dessa forma, conseguimos treinar e validar o modelo com diversas combinações de imagens do dataset, e observar se existe, por exemplo, overfitting.

Realizamos a validação cruzada com 5 folds e 10 epochs por fold. Além disso, foram usados batches de tamanho 32 para treinar o modelo.

O tamanho do batch foi obtido quando ainda era usado o MSE como função de loss, o que explica a acurácia baixa. Foram realizados testes com tamanhos de batch iguais a 16, 32, 64 e 128, obtendo as seguintes acurácias:

<table>
  <tr>
    <th>Tamanho do Batch</th>
    <th>Acurácia</th>
  </tr>
  <tr>
    <td>16</td>
    <td>0.629</td>
   
  </tr>
  <tr>
    <td>32</td>
    <td>0.710</td>
    
  </tr>
    <tr>
    <td>64</td>
    <td>0.613</td>
    
  </tr>
    <tr>
    <td>128</td>
    <td>0.661</td>
</table>

Como o tamanho de batch igual a 32 foi o que teve a melhor acurácia, ele foi o escolhido.

In [ ]:
epochs = 10
batch_size = 32
scale_logits = 10

X_data = X_norm
y_data = y_total
y_hinge = 2 * y_data - 1

NUM_FOLDS = 5
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

# Armazena métricas de todos os folds
all_fold_accuracies = []
all_fold_precisions = []
all_fold_recalls = []
all_fold_f1s = []

for fold, (train_ind, valid_ind) in enumerate(skf.split(X_data, y_data), 1):
    print(f"\nFold: {fold}/{NUM_FOLDS}")

    X_train, X_valid = X_data[train_ind], X_data[valid_ind]
    y_train, y_valid = y_hinge[train_ind], y_hinge[valid_ind]

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    y_valid_tensor = torch.tensor(y_valid, dtype=torch.float32).unsqueeze(1)

    num_samples = X_train_tensor.shape[0]
    num_batches = int(np.ceil(num_samples / batch_size))

    for epoch in range(epochs):
        permutation = torch.randperm(num_samples)
        X_train_shuffled = X_train_tensor[permutation]
        y_train_shuffled = y_train_tensor[permutation]

        epoch_loss = 0

        for batch_id in range(num_batches):
            # Seleciona os indices de incio e fim de cada barch
            start = batch_id * batch_size
            end = min(start + batch_size, num_samples)

            # Filtra o conjunto de dados de cada batch
            X_batch = X_train_shuffled[start:end]
            y_batch = y_train_shuffled[start:end]

            # Evita acumular gradientes de outros batches
            optimizer.zero_grad()

            # Faz as previsões
            outputs = torch.stack([circuit(x, weights_basic, weights_strongly) for x in X_batch]).unsqueeze(1)

            # Multiplicação do output por uma escala para ajudar a minimizar a perda
            logits = outputs * scale_logits

            # Calcula o loss
            loss = hinge_loss(logits, y_batch)

            # Calcula a direção que deve mudar para reduzir o erro
            loss.backward()

            # Otimizador atualiza os pesos do modelo
            optimizer.step()

            # Acumula o loss de cada batch
            epoch_loss += loss.item() * (end - start)

        # Calcula a média de loss dos batches para calcular o loss da epoch
        epoch_loss /= num_samples
        print(f"Epoch {epoch + 1}/{epochs}: loss = {epoch_loss:.4f}")

    # Avaliação no conjunto de validação
    with torch.no_grad():
        outputs_valid = torch.stack([circuit(x, weights_basic, weights_strongly) for x in X_valid_tensor]).unsqueeze(1)
        logits_valid = outputs_valid * scale_logits
        preds_valid_binary = (logits_valid.numpy() > 0).astype(int)

    # Converte a previsão para 0 ou 1
    y_valid_binary = ((y_valid_tensor + 1) / 2).numpy().astype(int)

    acc_valid = accuracy_score(y_valid_binary, preds_valid_binary)
    prec_valid = precision_score(y_valid_binary, preds_valid_binary)
    rec_valid = recall_score(y_valid_binary, preds_valid_binary)
    f1_valid = f1_score(y_valid_binary, preds_valid_binary)

    all_fold_accuracies.append(acc_valid)
    all_fold_precisions.append(prec_valid)
    all_fold_recalls.append(rec_valid)
    all_fold_f1s.append(f1_valid)

    print(f"Acurácia no fold {fold}: {acc_valid:.3f}")
    print(f"Precisão no fold {fold}: {prec_valid:.3f}")
    print(f"Recall no fold {fold}: {rec_valid:.3f}")
    print(f"F1-score no fold {fold}: {f1_valid:.3f}")

    # Matriz de confusão
    cm = confusion_matrix(y_valid_binary, preds_valid_binary)
    print(f"Matriz de confusão no fold {fold}:\n{cm}")

# Resultados finais
mean_acc = np.mean(all_fold_accuracies)
std_acc = np.std(all_fold_accuracies)

mean_prec = np.mean(all_fold_precisions)
mean_rec = np.mean(all_fold_recalls)
mean_f1 = np.mean(all_fold_f1s)

print("\n=== Resultados finais ===")
print(f"Acurácia média: {mean_acc:.3f} ± {std_acc:.3f}")
print(f"Precisão média: {mean_prec:.3f}")
print(f"Recall médio:   {mean_rec:.3f}")
print(f"F1-score médio: {mean_f1:.3f}")



Fold: 1/5
Epoch 1/10: loss = 0.6483
Epoch 2/10: loss = 0.4260
Epoch 3/10: loss = 0.2262
Epoch 4/10: loss = 0.0946
Epoch 5/10: loss = 0.0684
Epoch 6/10: loss = 0.0570
Epoch 7/10: loss = 0.0432
Epoch 8/10: loss = 0.0529
Epoch 9/10: loss = 0.0417
Epoch 10/10: loss = 0.0348
Acurácia no fold 1: 0.935
Precisão no fold 1: 0.933
Recall no fold 1: 0.933
F1-score no fold 1: 0.933
Matriz de confusão no fold 1:
[[30  2]
 [ 2 28]]

Fold: 2/5
Epoch 1/10: loss = 0.0658
Epoch 2/10: loss = 0.0592
Epoch 3/10: loss = 0.0593
Epoch 4/10: loss = 0.0542
Epoch 5/10: loss = 0.0699
Epoch 6/10: loss = 0.0570
Epoch 7/10: loss = 0.0551
Epoch 8/10: loss = 0.0623
Epoch 9/10: loss = 0.0472
Epoch 10/10: loss = 0.0421
Acurácia no fold 2: 0.984
Precisão no fold 2: 1.000
Recall no fold 2: 0.967
F1-score no fold 2: 0.983
Matriz de confusão no fold 2:
[[31  0]
 [ 1 29]]

Fold: 3/5
Epoch 1/10: loss = 0.0536
Epoch 2/10: loss = 0.0472
Epoch 3/10: loss = 0.0525
Epoch 4/10: loss = 0.0414
Epoch 5/10: loss = 0.0441
Epoch 6/10: l

# **Model V-2**

## Download do Dataset Pré-Processado



Antes de executar o modelo, é recomendado baixar o arquivo .zip do dataset criado em Cálculo de Parâmetro na parte de Pré-Processamento, usando o seguinte link: [params-dataset](https://drive.google.com/drive/folders/1Fup_vWfSOut8yUA7amQqspfPILGhBX_u?usp=sharing)

Após baixar o arquivo compactado, faça upload dele para a sua sessão do Colab e em seguida execute a célula abaixo.

In [ ]:
!unzip params-dataset.zip

Archive:  params-dataset.zip
  inflating: dataset_params.csv      


Se não ocorreu nenhum erro e foi criado o diretório params-dataset na sua sessão do Colab, você está pronto para executar o modelo!

## Carregamento e Normalização dos Dados

Antes de irmos ao Circuito Quântico Variacional, devemos pegar os dados brutos do arquivo, normalizá-los e prepará-los para serem carregados no circuito. Para isso, devemos realizar o seguinte trecho de código.

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Carrega o dataset salvo em um arquivo CSV
data = pd.read_csv('dataset_params.csv')

#
df_ai_gen_1 = data[data['ai_gen'] == 1]
df_ai_gen_0 = data[data['ai_gen'] == 0]

# Concatena os dois subconjuntos de dados balanceados em um único DataFrame, reindexando as linhas sequencialmente
data = pd.concat([df_ai_gen_1, df_ai_gen_0], ignore_index=True)

# Matriz de atributos (features) extraídos das imagens
X = data[['fft_mean',
          'fft_total_energy',
          'fft_max',
          'fft_std',
          'fft_entropy',
          'std_hue',
          'std_chroma',
          'std_lightness',
          'std_saturation']].values

# Vetor de rótulos (0 ou 1)
y = data['ai_gen'].values

# O 'AmplitudeEmbedding' da PennyLane exige que o vetor de entrada seja um vetor de amplitude normalizado e com norma 1. Dito isso, usamos
# [0, 1] para garantir que o vetor de entrada seja compatível com o espaço de estados físicos de um sistema quântico simulado (normalização)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)

## Circuito Quântico Variacional

Agora que os dados estão prontos para serem inseridos no circuito, podemos definir o circuito variacional propriamente dito.

Caso o *PennyLane* não esteja instalado na sua sessão do Colab, execute a célula abaixo.

In [ ]:
!pip install pennylane

In [ ]:
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


In [ ]:
# Definições iniciais
n_qubits = 4
dev = qml.device("default.qubit", wires=NUM_QUBITS)

n_layers_basic = 1
n_layers_strong = 1

# Definição do circuito variacional
@qml.qnode(dev)
def circuit(params_basic, params_strong, x):
    # Aplicação do Embedding Amplitude preenchendo com 0 as amplitudes vagas
    qml.templates.AmplitudeEmbedding(x, wires=range(NUM_QUBITS), normalize=True, pad_with=0.0)
    # Aplica templates
    qml.templates.StronglyEntanglingLayers(params_strong, wires=range(NUM_QUBITS))
    qml.templates.BasicEntanglerLayers(params_basic, wires=range(NUM_QUBITS))
    # Retorna o valor esperado
    return qml.expval(qml.PauliZ(0))

# Funções auxiliares
# Transforma os valores esperados retornados pelo circuito em 0 ou 1, facilitando a classificação binária
def predict(params_basic, params_strong, x):
    preds = [circuit(params_basic, params_strong, xi) for xi in x]
    preds = np.sign(np.array(preds))
    return (preds + 1) // 2

# Mede quão boa é a previsão quântica comparada ao valor real
def BinaryCrossEntropy(params_basic, params_strong, x, y):
    preds = [circuit(params_basic, params_strong, xi) for xi in x]
    probs = (np.array(preds) + 1) / 2
    eps = 1e-10
    probs = np.clip(probs, eps, 1 - eps)
    # Se o rótulo for 1, o termo ativo é np.log(probs), penalizando as previsões próximas de 0. Por outro lado,
    # se o rótulo for 0, o termo ativo é np.log(1 - probs), penalizando as previsões próximas de 1
    loss = - (y * np.log(probs) + (1 - y) * np.log(1 - probs))
    return np.mean(loss)

### Embedding

Executando uma validação cruzada nos quatro modelos (embedding na amplitude com 4 qubits e embedding no ângulo em RX, RY e RZ com 9 qubits), foram obtidas as seguintes métricas:

<table>
  <tr>
    <th>Embedding</th>
    <th>Acurácia Média</th>
    <th>Desvio Padrão Acurácia</th>
    <th>Precisão Média</th>
    <th>Recall Média</th>
    <th>F1-Score Média</th>
  </tr>
  <tr>
    <td>Amplitude</td>
    <td>0.954</td>
    <td>0.018</td>
    <td>0.994</td>
    <td>0.913</td>
    <td>0.951</td>
  </tr>
  <tr>
    <td>Ângulo RX</td>
    <td>0.630</td>
    <td>0.047</td>
    <td>0.651</td>
    <td>0.540</td>
    <td>0.589</td>
  </tr>
  <tr>
    <td>Ângulo RY</td>
    <td>0.934</td>
    <td>0.014</td>
    <td>1.000</td>
    <td>0.867</td>
    <td>0.928</td>
  </tr>
  <tr>
    <td>Ângulo RZ</td>
    <td>0.503</td>
    <td>0.013</td>
    <td>0.197</td>
    <td>0.400</td>
    <td>0.264</td>
  </tr>
</table>

Como é possível observar, o modelo com Embedding na Amplitude performa melhor que as demais. Devido a isso, optamos pelo Embedding na Amplitude para o circuito final. Além disso, esse método se torna mais vantajoso, uma vez que temos um menor número de qubits (4) em comparação com o Embedding no ângulo (9), tornando o circuito mais simples e rápido de simular.

### Layers

Após algumas pesquisas, encontramos que os templates de layers `BasicEntanglerLayers` e `StronglyEntanglingLayers` do *PennyLane* eram boas escolhas para classificação. Por causa disso, testamos várias combinações dos dois, e em um treinamento com tamanho de batch igual a 32 e 20 epochs conseguimos as seguintes acurácias:

<table>
  <tr>
    <th></th>
    <th>BasicEntanglerLayers</th>
    <th>StronglyEntanglingLayers</th>
    <th>Acurácia</th>
  </tr>
  <tr>
    <td>Teste 1</td>
    <td>0</td>
    <td>2</td>
    <td>0.937</td>
  </tr>
  <tr>
    <td>Teste 2</td>
    <td>0</td>
    <td>3</td>
    <td>0.905</td>
  </tr>
    <tr>
    <td>Teste 3</td>
    <td>2</td>
    <td>0</td>
    <td>0.827</td>
  </tr>
    <tr>
    <td>Teste 4</td>
    <td>3</td>
    <td>0</td>
    <td>0.866</td>
  </tr>
    <tr>
    <td>Teste 5</td>
    <td>1 (primeiro)</td>
    <td>1 (segundo)</td>
    <td>0.951</td>
  </tr>
  </tr>
    <tr>
    <td>Teste 6</td>
    <td>1 (segundo)</td>
    <td>1 (primeiro)</td>
    <td>0.954</td>
  </tr>
</table>

Apesar de não ter uma diferença significativa entre os dados, como o Teste 6 foi o com melhor performace, optamos por usar as layers testadas nele.

### Otimizador

Foram testados os otimizadores Adam e Nesterov usando as suas respectivas implementações do *Pennylane*.
Foram realizados treinamentos sob as mesmas circunstânceas com ambos (tamanho de batch igual a 32 e 20 epochs) e usando o circuito com as layers escolhidas anteriormente, de modo que o Adam atingiu $94.7\%$ de acurácia, já o Nesterov, $95.4\%$. Apesar da pequena diferença, optamos pelo otimizador Nesterov neste modelo.

### Loss

Inicialmente usamos a função de MSE (Mean Squared Error) como função de loss e a acurácia final atingia cerca de $65\%$ a $80\%$. Entretanto, após algumas pesquisas, encontramos que essa não era uma boa função para classificadores, sendo mais adequada para modelos de regressão. Portanto, optamos por testar a Hinge Loss e a Binary Cross Entropy, que eram mais adequadas para o nosso propósito. Feitos os testes sob as mesmas circunstâncias, observamos que as acurácias utilizando a Binary Cross Entropy foram melhores, devido a isso, optamos por utilizá-la, apesar de deixar o treinamento mais demorado.

## Treinamento e Avaliação do Modelo

Para a avaliação final do modelo, optamos por realizá-la usando validação cruzada, já que, dessa forma, conseguimos treinar e validar o modelo com diversas combinações de imagens do dataset, e observar se existe, por exemplo, overfitting.

Realizamos a validação cruzada com 5 folds e 20 epochs por fold. Além disso, foram usados batches de tamanho 32 para treinar o modelo.

Foram realizados testes com tamanhos de batch iguais a 16, 32, 64 e 128, obtendo as seguintes acurácias:

<table>
  <tr>
    <th>Tamanho do Batch</th>
    <th>Acurácia</th>
  </tr>
  <tr>
    <td>16</td>
    <td>0.958</td>
   
  </tr>
  <tr>
    <td>32</td>
    <td>0.954</td>
    
  </tr>
    <tr>
    <td>64</td>
    <td>0.905</td>
    
  </tr>
    <tr>
    <td>128</td>
    <td>0.735</td>
</table>

Apesar do batch com tamanho 16 apresentar resultados ligeiramente melhores que o de 32, escolhemos seguir com o batch de 32 para "padronizar" o treinamento de ambos os medelos.

In [ ]:
# Inicialização de listas para armazenar métricas
all_fold_accuracies = []
all_fold_precisions = []
all_fold_recalls = []
all_fold_f1s = []

# Cria o objeto Skf que divide o conjunto de dados em 5 folds preservando a proporção, evitando, assim, um viés
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(skf.split(X_scaled, y), 1):
    print(f"\nFold {fold}")

    # Divisão treino/teste para o fold atual
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Inicializa parâmetros com valores próximos de 0, com requires_grad=True permitindo a diferenciação automática,
    # garantindo que o circuito começe próximo do estado identidade, facilitando o aprendizado nos primeiros passos
    params_basic = qml.numpy.array(0.01 * np.random.randn(n_layers_basic, n_qubits), requires_grad=True)
    params_strong = qml.numpy.array(0.01 * np.random.randn(n_layers_strong, n_qubits, 3), requires_grad=True)

    # Inicializa otimizador
    opt = qml.NesterovMomentumOptimizer(stepsize=0.01, momentum=0.9)

    # Parâmetros
    epochs = 20
    batch_size = 32

    for epoch in range(epochs):
        indices = np.random.permutation(len(X_train))
        # Embaralha os dados antes de dividir em batches, garantindo que cada minibatch seja representativo e variado
        X_train_shuffled = X_train[indices]
        y_train_shuffled = y_train[indices]

        # Divide os dados em minibatches de tamanho 32
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train_shuffled[i:i+batch_size]
            y_batch = y_train_shuffled[i:i+batch_size]

            # Atualiza os parâmetros usando a função BinaryCrossEntropy como função de custo
            params_basic, params_strong = opt.step(
                lambda pb, ps: BinaryCrossEntropy(pb, ps, X_batch, y_batch),
                params_basic, params_strong
            )

        train_loss = BinaryCrossEntropy(params_basic, params_strong, X_train, y_train)
        print(f"Epoch {epoch+1}: Loss = {train_loss:.4f}")

    # Avaliação no conjunto de teste
    y_pred = predict(params_basic, params_strong, X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred, labels=[0, 1])

    all_fold_accuracies.append(acc)
    all_fold_precisions.append(prec)
    all_fold_recalls.append(rec)
    all_fold_f1s.append(f1)

    # % de acertos totais
    print(f"Acurácia no fold {fold}: {acc:.3f}")
    # Entre os que o modelo previu como 1, quantos eram realmente 1
    print(f"Precisão no fold {fold}: {prec:.3f}")
    # Entre os que realmente eram 1, quantos foram detectados
    print(f"Recall no fold {fold}: {rec:.3f}")
    # Média harmônica entre precisão e recall
    print(f"F1-score no fold {fold}: {f1:.3f}")
    print(f"Matriz de confusão no fold {fold}:\n{cm}")

# Resultados finais
print(f"\nMédia da Acurácia: {np.mean(all_fold_accuracies) * 100:.3f}% ± {np.std(all_fold_accuracies) * 100:.3f}%")
print(f"Precisão média: {np.mean(all_fold_precisions):.3f}")
print(f"Recall médio:   {np.mean(all_fold_recalls):.3f}")
print(f"F1-score médio: {np.mean(all_fold_f1s):.3f}")


 Fold 1
Epoch 1: Loss = 0.6511
Epoch 2: Loss = 0.5326
Epoch 3: Loss = 0.4597
Epoch 4: Loss = 0.3990
Epoch 5: Loss = 0.3740
Epoch 6: Loss = 0.3670


# **Conclusão**

Observa-se que os modelos avaliados alcançam acurácias elevadas, demonstrando consistência nos resultados. No entanto, à medida que aumentamos a complexidade das imagens, pode ser necessário realizar ajustes nos modelos. O primeiro modelo se destaca por trabalhar com imagens mais diversas, já que reduz suas dimensões para vetores de tamanho 8, o que contribui para uma boa performance em cenários mais simples. Por outro lado, o segundo modelo mostra-se mais escalável e, com a adição de alguns parâmetros, é capaz de realizar predições satisfatórias mesmo em contextos mais complexos.

# **Referências**
- GeeksforGeeks. (n.d.). Principal Component Analysis (PCA). Disponível em: https://www.geeksforgeeks.org/principal-component-analysis-pca/

- Medium – Data Science. (2019). Principal Component Analysis Made Easy: A Step-by-Step Tutorial. Disponível em: https://medium.com/data-science/principal-component-analysis-made-easy-a-step-by-step-tutorial-184f295e97fe

- Huynh, N. (n.d.). Understanding Loss Functions for Classification. Medium. Disponível em: https://medium.com/@nghihuynh_37300/understanding-loss-functions-for-classification-81c19ee72c2a

- Pedregosa, F. et al. (n.d.). 3.1. Cross-validation: Evaluating Estimator Performance. scikit-learn. Disponível em: https://scikit-learn.org/stable/modules/cross_validation.html

- PennyLane Documentation. (n.d.). Introduction to Templates. Disponível em: https://docs.pennylane.ai/en/stable/introduction/templates.html

- Wang, J., Webster, M., & Joyce, T. (2023). Colour Statistics of Images Created by Generative AI. ResearchGate. Disponível em: https://www.researchgate.net/publication/387287689_Colour_statistics_of_images_created_by_generative-AI

- Alam, M., Muneer, A., & Woo, W. (2024). UGAD: Universal Generative AI Detector Utilizing Frequency Fingerprint. ACM Digital Library. Disponível em: https://dl.acm.org/doi/10.1145/3627673.3680085

- Yuan, L., Li, X., Zhang, Y., Zhang, J., Li, H., & Gao, X. (2025). MLEP: Multi-granularity Local Entropy Patterns for Universal AI-generated Image Detection. arXiv preprint arXiv:1911.06465. Disponível em: https://arxiv.org/abs/1911.06465
